📈 Stock Price Prediction
===================================

✍️ Author: Shah Ahmad Noorani


🔗 Connect with Me:

💼 LinkedIn:
https://www.linkedin.com/in/shah-ahmad-noorani-5804b0278/

💻 GitHub:
https://github.com/shahahmad-dev

📊 Kaggle:
https://www.kaggle.com/shahahmadnoorani

**`Stock Market Data scrapping in Python`**

In [ ]:
# imporot libraries
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns
import yfinance as yf
import datetime as dt
from datetime import date, timedelta
import plotly.graph_objects as go
import plotly.express as px

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [ ]:
# define the date to fetch the data
# define the dates to fetch data
today = date.today()
d1 = today.strftime("%Y-%m-%d")
end_date = d1
d2 = date.today() - timedelta(days=365)
d2 = d2.strftime("%Y-%m-%d")
start_date = d2
print(f'The data is started from {start_day} to {end_day}')

In [ ]:
# deifne the ticker
ticker = 'GOOGL'

df = yf.download(ticker, start=start_date, end=end_date, progress=False)

In [ ]:
# make date numaric
df.insert(0, "Date", df.index, True)
# df["Date"] = df.index
df.reset_index(drop=True, inplace=True)

df.columns = df.columns.get_level_values(0)

In [ ]:
#  reset columns sequence
df = df[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]

In [ ]:
# remove the name column
df.columns.name = None

In [ ]:
df.describe()

In [ ]:
fig = px.line(df, x='Date', y=df.columns, title='Stock Price')
fig.show()

In [ ]:
# select columns
df1 = df[['Date', 'Close']]
df1.head()

In [ ]:
# plot the to see how it look like
fig = px.line(df1, x='Date', y='Close', title='Google Stock Price')
fig.show()

In [ ]:
# Check the data is Statinority or not
from statsmodels.tsa.stattools import adfuller
def check_stationarity(df):
  result = adfuller(df)
  print('ADF Statistic: %f'% result[0])
  print('P-value: %f'% result[0])
  if result[1]<= 0.05:
    print('Data is Stationary')
  else:
    print("Data isn't Stationary")

In [ ]:
check_stationarity(df['Close'])

In [ ]:
# Decompose the data to see the trend
from statsmodels.tsa.seasonal import seasonal_decompose
decompose = seasonal_decompose(df['Close'], model='additive', period=30)
decompose.plot()
plt.show()